<a href="https://colab.research.google.com/github/subeans/security_project/blob/main/dbscan_official.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot  as plt
import seaborn as sns

In [ ]:
from google.colab import drive 
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
dataset = pd.read_csv("/content/drive/MyDrive/security_project/04_hashed.csv")

dataset.columns

FileNotFoundError: ignored

In [ ]:
def select_unique_Rdate(dataset):
  time_list=[]
  for i in range(len(dataset['Rdate'])):
    time_list.append(dataset.iloc[i]['Rdate'])
  unique_time = set(time_list)
  unique_time = sorted(unique_time)
  print(len(unique_time)) # 24218
  return unique_time 

In [ ]:
# Rdate의 중복시간제거
unique_time = select_unique_Rdate(dataset)

In [ ]:
""" 
IPv4 ,각 그룹을 균등화하고 길이가 일치하도록 0 추가 
""" 
def transform_ip(ip): 
  IPV4_GROUP_LENGTH = 3 # IPv4의 각 그룹은 이 길이입니다.
  # IPv4 주소 
  groups = ip.split( "." ) 
  equalize_group_length = "".join( map( lambda group: group.zfill(3), groups )) 
  return equalize_group_length 

"""
Converts the ipAddress column of pandas DataFrame df, to one-hot
Also returns the encoder used
"""
from sklearn.preprocessing import OneHotEncoder

def one_hot_ip(df):
  df['src_ip'] = df.src_ip.apply(lambda ip : transform_ip(ip))
  df['dst_ip'] = df.dst_ip.apply(lambda ip : transform_ip(ip))
  return df

In [ ]:
feature = one_hot_ip(dataset)

In [ ]:
# src_country / dst_country 를 사용하는 경우 
def labelEncoder(dataset):
  from sklearn.preprocessing import LabelEncoder
  job_encode = LabelEncoder()
  dataset['src_country'] = job_encode.fit_transform(dataset['src_country'])
  dataset['dst_country'] = job_encode.fit_transform(dataset['dst_country'])
  
  return dataset

In [ ]:
feature_ = labelEncoder(feature)

In [ ]:
# #############################################################################
# Compute DBSCAN
def dbscan(feature_, min_samples, eps):
  from sklearn.preprocessing import StandardScaler
  scaler = StandardScaler()

  # 사용할 feature 선정 
  feature = feature_[ ['src_ip', 'dst_ip', 'src_port', 'dst_port', 'Action']]
  scaler.fit(feature)
  feature_trans = scaler.transform(feature)

  # 파라미터 변경 
  model = DBSCAN(eps=eps, min_samples=min_samples)
  predict = pd.DataFrame(model.fit_predict(feature_trans))
  predict.columns = ['predict']
  r = pd.concat([feature_,predict],axis=1)

  labels = model.labels_

  n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
  n_noise_ = list(labels).count(-1)

  print("Dataset Size : ",len(feature))
  print("Estimated number of clusters: %d" % n_clusters_)
  print("Estimated number of noise points: %d" % n_noise_)
  return r, labels

In [ ]:
# draw
def pair_plot(r):
  sns.pairplot(r,hue='predict')
  plt.show()


In [ ]:
unique_time = select_unique_Rdate(train_dataset)

In [ ]:
# window size 만큼의의 데이터 추출
window_size = 100
dbscan_final = pd.DataFrame()
for i in range(0,len(unique_time)-window_size,50):
  print(i)
  window_df = pd.DataFrame()
  for j in range(window_size):
    # 특정시간대의 데이터만 추출
    select_time = feature_[feature_['Rdate']==unique_time[i+j]]
    window_df = pd.concat([window_df,select_time])
    
  # 추출된 특정시간대의 데이터만가지고 DBSCAN 진행
  dbscan_result,labels = dbscan(window_df, 5, 100)
  anomaly_data = dbscan_result[dbscan_result['predict']==-1]
  dbscan_final = pd.concat([anomaly_data,dbscan_final])
  print()
  # pair_plot(dbscan_result)
  # break